In [53]:
import requests
import os
response = requests.get(
    f"http://localhost:5000/v1/llm/model-endpoints",
    headers={"Content-Type": "application/json"},
    auth=requests.auth.HTTPBasicAuth(os.getenv("LAUNCH_API_KEY"), ""),
    # timeout=DEFAULT_NETWORK_TIMEOUT_SEC,
)

print(response.status_code)
print(response.json())

200
{'model_endpoints': []}


In [39]:
import os

from pydantic import BaseModel

from launch_internal import get_launch_client  # type: ignore


class MyRequestSchema(BaseModel):
    x: int
    y: str


class MyResponseSchema(BaseModel):
    __root__: int


api_key = os.getenv("LAUNCH_API_KEY")
env = "training"
gateway_endpoint = "http://localhost:5000"
launch_client = get_launch_client(api_key=api_key, env=env, gateway_endpoint=gateway_endpoint)



In [71]:
launch_client.list_model_bundles_v2()

ListModelBundlesV2Response(model_bundles=[ModelBundleV2Response(id='bun_ciqp58jqqncg03vld5u0', name='test-streaming-bundle', metadata={}, created_at=datetime.datetime(2023, 7, 17, 19, 12, 34, 640168, tzinfo=datetime.timezone.utc), model_artifact_ids=[], schema_location='s3://scale-ml/scale-launch/model_bundles/646c085a6b38de00808148bb/705ac616-59df-4382-8c69-83accb8c0bd8', flavor=StreamingEnhancedRunnableImageFlavor(repository='ghcr.io/huggingface/text-generation-inference', tag='0.9.1', command=['text-generation-launcher', '--model-id', 'tiiuae/falcon-7b', '--num-shard', '4', '--port', '5005', '--hostname', '::'], env={'TEST_KEY': 'test_value'}, protocol='http', readiness_initial_delay_seconds=30, flavor=<ModelBundleFlavorType.STREAMING_ENHANCED_RUNNABLE_IMAGE: 'streaming_enhanced_runnable_image'>, streaming_command=['text-generation-launcher', '--model-id', 'tiiuae/falcon-7b', '--num-shard', '4', '--port', '5005', '--hostname', '::']))])

In [66]:
launch_client.list_model_endpoints()

[SyncEndpoint(name='test-wsong', bundle_name='test-streaming-bundle', status='READY', resource_state='{"cpus": "32", "gpus": 4, "memory": "185Gi", "gpu_type": "nvidia-ampere-a10", "storage": "200G", "optimize_costs": false}', deployment_state='{"min_workers": 1, "max_workers": 3, "per_worker": 10, "available_workers": 0, "unavailable_workers": 1}', endpoint_type='sync', metadata='{}')]

In [72]:
launch_client.delete_model_endpoint("test-wsong")

True

In [ ]:
class MyRequestSchema(BaseModel):
    x: int
    y: str

class MyResponseSchema(BaseModel):
    __root__: int


def my_load_predict_fn(model):
    def returns_model_of_x_plus_len_of_y(x: int, y: str) -> int:
        """MyRequestSchema -> MyResponseSchema"""
        assert isinstance(x, int) and isinstance(y, str)
        return model(x) + len(y)

    return returns_model_of_x_plus_len_of_y


def my_load_model_fn():
    def my_model(x):
        return x * 2

    return my_model

BUNDLE_PARAMS = {
    "model_bundle_name": "test-bundle",
    "load_model_fn": my_load_model_fn,
    "load_predict_fn": my_load_predict_fn,
    "request_schema": MyRequestSchema,
    "response_schema": MyResponseSchema,
    "requirements": ["pytest==7.2.1", "numpy"],  # list your requirements here
    "pytorch_image_tag": "1.7.1-cuda11.0-cudnn8-runtime",
}

client = launch_client(api_key=os.getenv("LAUNCH_API_KEY"))
client.create_model_bundle_from_callable_v2(**BUNDLE_PARAMS)


In [13]:
bundle_name = "test-streaming-bundle"
hf_model_name = "tiiuae/falcon-7b"
num_shards = 4
command = [
    "text-generation-launcher",
    "--model-id",
    hf_model_name,
    "--num-shard",
    str(num_shards),
    "--port",
    "5005",
    "--hostname",
    "::",
]

BUNDLE_PARAMS = {
    "model_bundle_name": bundle_name,
    "request_schema": MyRequestSchema,
    "response_schema": MyResponseSchema,
    "repository": "ghcr.io/huggingface/text-generation-inference",
    "tag": "0.9.1",
    "command": command,  # optional; if provided, will also expose the /predict endpoint
    "predict_route": "/predict",
    "healthcheck_route": "/readyz",
    "streaming_command": command,  # required
    "streaming_predict_route": "/stream",
    "env": {
        "TEST_KEY": "test_value",
    },
    "readiness_initial_delay_seconds": 30,
}

# client = LaunchClient(api_key=os.getenv("LAUNCH_API_KEY"), endpoint='https://launch.ml-internal.scale.com')
# print("submitting request")
# resp = launch_client.create_model_bundle_from_streaming_enhanced_runnable_image_v2(**BUNDLE_PARAMS)
# print("got resp")
# print(resp)



In [74]:
endpoint_payload = {
    "endpoint_name": "test-wsong",
    "endpoint_type": "sync",
    "update_if_exists": True,
    "model_bundle": bundle_name,
    "min_workers": 1,
    "max_workers": 3,
    "per_worker": 10,
    "memory": "60Gi",
    "storage": "20G",
    "cpus": 32,
    "gpus": 4,
    "gpu_type": "nvidia-ampere-a10",
    "labels": {
        "team": "infra",
        "product": "llm_model_zoo",
    },
}


print(f"Creating endpoint with payload:\n{endpoint_payload}")
endpoint = launch_client.create_model_endpoint(**endpoint_payload)
print(f"Created endpoint: {endpoint}")

Creating endpoint with payload:
{'endpoint_name': 'test-wsong', 'endpoint_type': 'sync', 'update_if_exists': True, 'model_bundle': 'test-streaming-bundle', 'min_workers': 1, 'max_workers': 3, 'per_worker': 10, 'memory': '60Gi', 'storage': '20G', 'cpus': 32, 'gpus': 4, 'gpu_type': 'nvidia-ampere-a10', 'labels': {'team': 'infra', 'product': 'llm_model_zoo'}}
Created endpoint: SyncEndpoint <endpoint_name:test-wsong>


SyncEndpoint <endpoint_name:test-wsong-2>
